In [3]:
from data_reader import SimulationDataset, generate_metadata
from learned_simulator import Simulator
import json
import torch
import lightning as L

INPUT_SEQUENCE_LENGTH = 6
DEVICE = 'cpu'

ds = SimulationDataset(device=DEVICE, mode='test', window_size=6)
metadata = generate_metadata(ds,mode='train')

num_steps = metadata['sequence_length'] - INPUT_SEQUENCE_LENGTH


KeyboardInterrupt: 

In [17]:
checkpoint_path = "lightning_logs/version_16/checkpoints/epoch=0-step=10.ckpt"
simulator = Simulator.load_from_checkpoint(
    checkpoint_path,
    particle_dimension=2,
    node_in=30,
    edge_in=3,
    latent_dim=128,
    num_message_passing_steps=10,
    mlp_num_layers=2,
    mlp_hidden_dim=128,
    num_particle_types=9,
    particle_type_embedding_size=16,
    metadata=metadata,
    strategy='baseline',
    device='cpu',
    )

In [15]:
def eval_single_rollout(simulator, features, num_steps, device):
    initial_positions = features[:, 0:INPUT_SEQUENCE_LENGTH]
    ground_truth_positions = features[:, INPUT_SEQUENCE_LENGTH:]
    
    current_positions = initial_positions
    predictions = []
    for step in range(num_steps):
        next_position = simulator.predict_positions(
            current_positions,
            n_particles_per_example=features['n_particles_per_example'],
            particle_types=features['particle_type'],
        ) # (n_nodes, 2)
        # Update kinematic particles from prescribed trajectory.
        kinematic_mask = (features['particle_type'] == 3).clone().detach().to(device)
        next_position_ground_truth = ground_truth_positions[:, step]
        kinematic_mask = kinematic_mask.bool()[:, None].expand(-1, 2)
        next_position = torch.where(kinematic_mask, next_position_ground_truth, next_position)
        predictions.append(next_position)
        current_positions = torch.cat([current_positions[:, 1:], next_position[:, None, :]], dim=1)
    predictions = torch.stack(predictions) # (time, n_nodes, 2)
    ground_truth_positions = ground_truth_positions.permute(1,0,2)
    loss = (predictions - ground_truth_positions) ** 2
    output_dict = {
        'initial_positions': initial_positions.permute(1,0,2).cpu().numpy(),
        'predicted_rollout': predictions.cpu().numpy(),
        'ground_truth_rollout': ground_truth_positions.cpu().numpy(),
        'particle_types': features['particle_type'].cpu().numpy(),
    }
    return output_dict, loss

In [24]:
for features, labels in ds:
    print(features[:, INPUT_SEQUENCE_LENGTH:])



    break


tensor([], size=(361, 0, 2))
